## **Classification of Personality Influenced Emotions using subjective modeling approach  **

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

**Install** **dependencies**

In [ ]:
!pip install tensorflow_text
!pip install tensorflow-addons

**Install required packages**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow import keras
from random import shuffle
import random
import re
import string

**Ulitity functions**

In [ ]:
# utility functions
def data_loader(PATH, EMO_OR_SENTI):
    characters = ['Chandler','Joey','Monica','Phoebe','Rachel','Ross']
    df = pd.read_csv(PATH, sep='\t').fillna('Nan')
    for actor in characters:
        # get rows from the selected character
        single_df = df[df['Speaker_1'].apply(lambda x: actor in x)]
        # selected emotion to increase the samples of that particulat emotions
        if actor == 'Chandler':
            increase_emo = ['anger', 'disgust', 'fear']
        else:
            increase_emo = ['anger', 'disgust', 'fear', 'surprise', 'sadness', 'joy']
        for emo in increase_emo:
            selected_df = single_df[single_df['Emotion_3'].apply(lambda x: emo in x)]
            percent = int(len(selected_df)*25/100)
            random_int = random.sample(range(0, len(selected_df)), percent)
            for idx in random_int:
                row = selected_df.iloc[idx]
                sp1 = row['Speaker_1']
                sp2 = row['Speaker_2']
                per = row['Personality']
                # utterence
                utt_1 = row['Utterance_1']
                utt_2 = row['Utterance_2']
                utt_3 = row['Utterance_3']
                # emotions
                emo_1 = row['Emotion_1']
                emo_2 = row['Emotion_2']
                emo_3 = row['Emotion_3']
                # sentiments
                sent_1 = row['Sentiment_1']
                sent_2 = row['Sentiment_2']
                sent_3 = row['Sentiment_3']
                # new row
                row1 = {}
                row1['Speaker_1'] = sp1
                row1['Speaker_2'] = sp2
                row1['Personality'] = per
                row1['Utterance_1'] = utt_3
                row1['Utterance_2'] = utt_2
                row1['Utterance_3'] = utt_1
                row1['Emotion_1'] = emo_3
                row1['Emotion_2'] = emo_2
                row1['Emotion_3'] = emo_1
                row1['Sentiment_1'] = sent_3
                row1['Sentiment_2'] = sent_2
                row1['Sentiment_3'] = sent_1
                single_df = single_df.append(row1,ignore_index=True)
        emo_vad = []
        if EMO_OR_SENTI == 'Emotions':
            labels = single_df['Emotion_1'].values
            for y in labels:
                if y == 'anger':
                    emo_vad.append([-0.51, 0.59, 0.25])
                elif y == 'disgust':
                    emo_vad.append([-0.60, 0.35, 0.11])
                elif y == 'fear':
                    emo_vad.append([-0.62, 0.82, -0.43])
                elif y == 'joy':
                    emo_vad.append([0.81, 0.51, 0.46])
                elif y == 'neutral':
                    emo_vad.append([0.00, 0.00, 0.00])
                elif y == 'sadness':
                    emo_vad.append([-0.63, -0.27, -0.33])
                elif y == 'surprise':
                    emo_vad.append([0.40, 0.67, -0.13])
            one_hot = pd.get_dummies(labels)
        else:
            labels = single_df['Sentiment_3'].values
            one_hot = pd.get_dummies(labels)
        Utterance_1   = single_df['Utterance_1'].values
        Utterance_2   = single_df['Utterance_2'].values
        Utterance_3   = single_df['Utterance_3'].values
        personality_traits = single_df['Personality'].values
        personality = personality_traits[0]
        personality = personality.replace('[','').replace(']','')
        personality = personality.split(',')
        VAD = []
        if actor == 'Chandler':
            O = float(personality[0])
            C = float(personality[1])
            E = float(personality[2])
            A = float(personality[3])
            N = float(personality[4])
            P_v = (0.21*E) + (0.59*A) + (0.19*N)
            P_a = (0.15*O) + (0.30*A) - (0.57*N)
            P_d = (0.25*O) + (0.17*C) + (0.60*E) - (0.32*A)
            VAD.append(float("{0:.4f}".format(P_v)))
            VAD.append(float("{0:.4f}".format(P_a)))
            VAD.append(float("{0:.4f}".format(P_d)))
        elif actor == 'Joey':
            O = float(personality[0])
            C = float(personality[1])
            E = float(personality[2])
            A = float(personality[3])
            N = float(personality[4])
            P_v = (0.21*E) + (0.59*A) + (0.19*N)
            P_a = (0.15*O) + (0.30*A) - (0.57*N)
            P_d = (0.25*O) + (0.17*C) + (0.60*E) - (0.32*A)
            VAD.append(float("{0:.4f}".format(P_v)))
            VAD.append(float("{0:.4f}".format(P_a)))
            VAD.append(float("{0:.4f}".format(P_d)))
        elif actor == 'Monica':
            O = float(personality[0])
            C = float(personality[1])
            E = float(personality[2])
            A = float(personality[3])
            N = float(personality[4])
            P_v = (0.21*E) + (0.59*A) + (0.19*N)
            P_a = (0.15*O) + (0.30*A) - (0.57*N)
            P_d = (0.25*O) + (0.17*C) + (0.60*E) - (0.32*A)
            VAD.append(float("{0:.4f}".format(P_v)))
            VAD.append(float("{0:.4f}".format(P_a)))
            VAD.append(float("{0:.4f}".format(P_d)))
        elif actor == 'Phoebe':
            O = float(personality[0])
            C = float(personality[1])
            E = float(personality[2])
            A = float(personality[3])
            N = float(personality[4])
            P_v = (0.21*E) + (0.59*A) + (0.19*N)
            P_a = (0.15*O) + (0.30*A) - (0.57*N)
            P_d = (0.25*O) + (0.17*C) + (0.60*E) - (0.32*A)
            VAD.append(float("{0:.4f}".format(P_v)))
            VAD.append(float("{0:.4f}".format(P_a)))
            VAD.append(float("{0:.4f}".format(P_d)))
        elif actor == 'Rachel':
            O = float(personality[0])
            C = float(personality[1])
            E = float(personality[2])
            A = float(personality[3])
            N = float(personality[4])
            P_v = (0.21*E) + (0.59*A) + (0.19*N)
            P_a = (0.15*O) + (0.30*A) - (0.57*N)
            P_d = (0.25*O) + (0.17*C) + (0.60*E) - (0.32*A)
            VAD.append(float("{0:.4f}".format(P_v)))
            VAD.append(float("{0:.4f}".format(P_a)))
            VAD.append(float("{0:.4f}".format(P_d)))
        elif actor == 'Ross':
            O = float(personality[0])
            C = float(personality[1])
            E = float(personality[2])
            A = float(personality[3])
            N = float(personality[4])
            P_v = (0.21*E) + (0.59*A) + (0.19*N)
            P_a = (0.15*O) + (0.30*A) - (0.57*N)
            P_d = (0.25*O) + (0.17*C) + (0.60*E) - (0.32*A)
            VAD.append(float("{0:.4f}".format(P_v)))
            VAD.append(float("{0:.4f}".format(P_a)))
            VAD.append(float("{0:.4f}".format(P_d)))

        Utterance_1 = np.reshape(Utterance_1,[len(Utterance_1),1])
        Utterance_2 = np.reshape(Utterance_2,[len(Utterance_2),1])
        Utterance_3 = np.reshape(Utterance_3,[len(Utterance_3),1])
        VAD = np.reshape(VAD,[1, len(VAD)])
        vad_personalities = np.repeat(VAD,len(personality_traits), axis=0)
        if actor == 'Chandler':
            Chandler = np.concatenate((Utterance_1,Utterance_2,Utterance_3,vad_personalities,emo_vad,one_hot), axis=1)
        elif actor == 'Joey':
            Joey = np.concatenate((Utterance_1,Utterance_2,Utterance_3,vad_personalities,emo_vad,one_hot), axis=1)
        elif actor == 'Monica':
            Monica = np.concatenate((Utterance_1,Utterance_2,Utterance_3,vad_personalities,emo_vad,one_hot), axis=1)
        elif actor == 'Phoebe':
            Phoebe = np.concatenate((Utterance_1,Utterance_2,Utterance_3,vad_personalities,emo_vad,one_hot), axis=1)
        elif actor == 'Rachel':
            Rachel = np.concatenate((Utterance_1,Utterance_2,Utterance_3,vad_personalities,emo_vad,one_hot), axis=1)
        elif actor == 'Ross':
            Ross = np.concatenate((Utterance_1,Utterance_2,Utterance_3,vad_personalities,emo_vad,one_hot), axis=1)

    return Chandler,Joey,Monica,Phoebe,Rachel,Ross

# get minimum samples
def min_samples(data):
  length = []
  for i in range(len(data)):
    samples = len(data[i])
    length.append(samples)
  return min(length)

# get 1D vector
def reduce_dimension(labels,logits):
    y_true = []
    y_pred = []
    for i in range(len(labels)):
        batch_labels = labels[i]
        batch_logits = logits[i]
        for j in range(len(batch_labels)):
            list_labels = batch_labels[j]
            list_logits = batch_logits[j]
            y_true.append(np.argmax(list_labels))
            y_pred.append(np.argmax(list_logits))
    return y_true,y_pred

# get normalize text
def normalize_text(array):
    norm_text = []
    for text in array:
      text = text.lower()
      # Remove punctuations
      exclude = set(string.punctuation)
      text = "".join(ch for ch in text if ch not in exclude)
      # Remove articles
      regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
      text = re.sub(regex, " ", text)
      # Remove extra white space
      text = " ".join(text.split())
      norm_text.append(text)
    return norm_text

**Download and split data into train, val , ans test set**

In [ ]:
# data file path
PATH = '/gdrive/My Drive/emotion_personality/New_PELD_Exp/Dyadic_PELD.tsv'

# select between 0:sentiments and 1:emotions
select_list = ['Emotions', 'Sentiments']
EMO_OR_SENTI = select_list[0]
Chandler,Joey,Monica,Phoebe,Rachel,Ross = data_loader(PATH,EMO_OR_SENTI)
 
# split each character samples into train and test sets
chandler_train, chandler_test = train_test_split(Chandler, random_state=42, test_size=0.1)
joey_train, joey_test = train_test_split(Joey, random_state=42, test_size=0.1)
monica_train, monica_test = train_test_split(Monica, random_state=42, test_size=0.1)
phoebe_train, phoebe_test = train_test_split(Phoebe, random_state=42, test_size=0.1)
rachel_train, rachel_test = train_test_split(Rachel, random_state=42, test_size=0.1)
ross_train, ross_test = train_test_split(Ross, random_state=42, test_size=0.1)

# split each character train samples into train and val
chandler_train, chandler_val = train_test_split(chandler_train, random_state=42, test_size=0.1)
joey_train, joey_val = train_test_split(joey_train, random_state=42, test_size=0.1)
monica_train, monica_val = train_test_split(monica_train, random_state=42, test_size=0.1)
phoebe_train, phoebe_val = train_test_split(phoebe_train, random_state=42, test_size=0.1)
rachel_train, rachel_val = train_test_split(rachel_train, random_state=42, test_size=0.1)
ross_train, ross_val = train_test_split(ross_train, random_state=42, test_size=0.1)


**Create BERT model**

In [ ]:
preprocessing_path = '/gdrive/My Drive/emotion_personality/New_PELD_Exp/bert_en_uncased_preprocess_3'
model_path = '/gdrive/My Drive/emotion_personality/New_PELD_Exp/bert_en_uncased_L-12_H-768_A-12_4'
# inputs shapes
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessing_layer = hub.KerasLayer(hub.load(preprocessing_path))
encoder_inputs = preprocessing_layer(text_input)
encoder = hub.KerasLayer(hub.load(model_path),trainable=True, name='BERT_encoder' )
outputs = encoder(encoder_inputs)
net = outputs['pooled_output']
bert_encoder = tf.keras.Model(text_input, net)

**Subjective models**

In [ ]:
# subjective models
class SubjectiveModel(tf.keras.Model):
  def __init__(self):
    super().__init__()
    initializer = tf.keras.initializers.GlorotUniform(seed=42)
    self.linear1 = tf.keras.layers.Dense(7, activation=tf.nn.softmax, kernel_initializer=initializer)
    self.dropout = tf.keras.layers.Dropout(0.5)
  def call(self, inputs, training=False):
    out = self.dropout(inputs)
    out = self.linear1(out, training=training)
    return out

# personality influence emotions
class PersonalityEmotions(tf.keras.Model):
  def __init__(self):
    super().__init__()
    initializer = tf.keras.initializers.GlorotUniform(seed=42)
    self.linear1 = tf.keras.layers.Dense(3, activation=None, kernel_initializer=initializer)
    self.linear2 = tf.keras.layers.Dense(3, activation=None, kernel_initializer=initializer)
    self.linear3 = tf.keras.layers.Dense(3, activation=None, kernel_initializer=initializer)
  def call(self, ini_emo, personality_vad, text_embd, training=False):
    # linear layer for the personality influenced VAD
    personality_influence = self.linear1(personality_vad,training=training)
    # linear layer for all the previous text encoding
    utter_emotions = self.linear2(text_embd,training=training)
    # get the target emotions
    target_emotions = ini_emo + utter_emotions * personality_influence
    # Tanh activation function
    target_emotions = tf.keras.activations.tanh(target_emotions)
    # linear layer for the target emotions
    target_emotions = self.linear3(target_emotions)
    # concatenate layer
    concatenate_matrix = tf.keras.layers.concatenate([personality_influence,target_emotions,text_embd])
    return concatenate_matrix

**Initialize all the subjective models, optimizer, loss function, and learning rate schedular** 

In [ ]:
# character 1
model_1 = SubjectiveModel()
personality_vad_1 = PersonalityEmotions()
# character 2
model_2 = SubjectiveModel()
personality_vad_2 = PersonalityEmotions()
# character 3
model_3 = SubjectiveModel()
personality_vad_3 = PersonalityEmotions()
# character 4
model_4 = SubjectiveModel()
personality_vad_4 = PersonalityEmotions()
# character 5
model_5 = SubjectiveModel()
personality_vad_5 = PersonalityEmotions()
# character 6
model_6 = SubjectiveModel()
personality_vad_6 = PersonalityEmotions()

# schedular
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  1e-4,
  decay_steps=10000,
  decay_rate=0.01,
  staircase=False)

# optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
# loss function
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False)

**Model hyperparameters**

In [ ]:
#The BERT authors recommend fine-tuning for 4 epochs over the following hyperparameter options: 
# batch sizes: 8, 16, 32, 64, 128. learning rates: 3e-4, 1e-4, 5e-5, 3e-5
epochs = 10
batch_size = 8

**Model training, validation, and testing**

In [ ]:
for epoch in range(epochs):
    print('*' * 50)
    print("\nStart of epoch %d" % (epoch,))
    print('*' * 50)
    # shuffle all the training samples for each character
    shuffle(chandler_train)
    shuffle(joey_train)
    shuffle(monica_train)
    shuffle(phoebe_train)
    shuffle(rachel_train)
    shuffle(ross_train)
    # concatenate all training samples
    train_data = []
    train_data.append(chandler_train)
    train_data.append(joey_train)
    train_data.append(monica_train)
    train_data.append(phoebe_train)
    train_data.append(rachel_train)
    train_data.append(ross_train)
    minimum_samples = min_samples(train_data)
    max_range = np.multiply(int(minimum_samples/batch_size),batch_size)
    chandler_loss = []
    joey_loss = []
    monica_loss = []
    phoebe_loss = []
    rachel_loss = []
    ross_loss = []
    for i in range(0,max_range,batch_size):
        for j in range(len(train_data)):
            X_train = train_data[j]
            X_utter1 = X_train[:,0]
            X_utter2 = X_train[:,1]
            X_p_vad = X_train[:,3:6]
            X_emo_vad = X_train[:,6:9]
            X_emo_labels = X_train[:,-7:]
            # select batch
            utter1 = X_utter1[i:i+batch_size]
            utter2 = X_utter2[i:i+batch_size]
            utter1 = normalize_text(utter1)
            utter2 = normalize_text(utter2)
            p_vad = X_p_vad[i:i+batch_size].astype('float32')
            cat_labels = X_emo_labels[i:i+batch_size].astype('float32')
            vad_labels = X_emo_vad[i:i+batch_size].astype('float32')
            # convert into tensors
            utter1 = tf.convert_to_tensor(utter1)
            utter2 = tf.convert_to_tensor(utter2)
            p_vad = tf.convert_to_tensor(p_vad)
            cat_labels = tf.convert_to_tensor(cat_labels)
            vad_labels = tf.convert_to_tensor(vad_labels)
            with tf.GradientTape(persistent=True) as tape:
                utter1_sementics = bert_encoder(utter1)
                utter2_sementics = bert_encoder(utter2)
                dialogue_sementics = tf.concat([utter1_sementics, utter2_sementics],1)
                if j == 0:
                    # personality vad
                    per_vad = personality_vad_1(vad_labels,p_vad, dialogue_sementics,training=True)
                    # logits
                    logits = model_1(per_vad,training=True)
                    # loss function  
                    loss_value = loss_fn(cat_labels, logits)
                    chandler_loss.append(loss_value)
                    trainables = model_1.trainable_weights  + personality_vad_1.trainable_weights + bert_encoder.trainable_weights    
                elif j == 1:
                    # personality vad
                    per_vad = personality_vad_2(vad_labels,p_vad, dialogue_sementics,training=True)
                    # logits
                    logits = model_2(per_vad,training=True)
                    # loss function  
                    loss_value = loss_fn(cat_labels, logits)
                    joey_loss.append(loss_value)
                    trainables = model_2.trainable_weights  + personality_vad_2.trainable_weights+ bert_encoder.trainable_weights
                elif j == 2:
                    # personality vad
                    per_vad = personality_vad_3(vad_labels,p_vad, dialogue_sementics,training=True)
                    # logits
                    logits = model_3(per_vad,training=True)
                    # loss function  
                    loss_value = loss_fn(cat_labels, logits)
                    monica_loss.append(loss_value)
                    trainables = model_3.trainable_weights  + personality_vad_3.trainable_weights+ bert_encoder.trainable_weights
                elif j == 3:
                    # personality vad
                    per_vad = personality_vad_4(vad_labels,p_vad,dialogue_sementics,training=True)
                    # logits
                    logits = model_4(per_vad,training=True)
                    # loss function 
                    loss_value = loss_fn(cat_labels, logits)
                    phoebe_loss.append(loss_value)
                    trainables = model_4.trainable_weights  + personality_vad_4.trainable_weights+ bert_encoder.trainable_weights
                elif j == 4:
                    # personality vad
                    per_vad = personality_vad_5(vad_labels,p_vad,dialogue_sementics,training=True)
                    # logits
                    logits = model_5(per_vad,training=True)
                    # loss function  
                    loss_value = loss_fn(cat_labels, logits)
                    rachel_loss.append(loss_value)
                    trainables = model_5.trainable_weights  + personality_vad_5.trainable_weights  + bert_encoder.trainable_weights
                elif j == 5:
                    # personality vad
                    per_vad = personality_vad_6(vad_labels,p_vad,dialogue_sementics,training=True)
                    # logits
                    logits = model_6(per_vad,training=True)
                    # loss function  
                    loss_value = loss_fn(cat_labels, logits)
                    ross_loss.append(loss_value)
                    trainables = model_6.trainable_weights  + personality_vad_6.trainable_weights+ bert_encoder.trainable_weights
            # Use the gradient tape to automatically retrieve
            # the gradients of the trainable variables with respect to the loss.
            grads = tape.gradient(loss_value, trainables)
            # Run one step of gradient descent by updating
            # the value of the variables to minimize the loss.
            optimizer.apply_gradients(zip(grads, trainables))
    print('-' * 20)
    print('Training Loss')
    print('-' * 20)
    print('Chandler Loss:', np.mean(chandler_loss))
    print('Joey Loss:', np.mean(joey_loss))
    print('Monica Loss:', np.mean(monica_loss))
    print('Phoebe Loss:', np.mean(phoebe_loss))
    print('Rachel Loss:', np.mean(rachel_loss))
    print('Ross Loss:', np.mean(ross_loss))
    # validation
    val_data = []
    val_data.append(chandler_val)
    val_data.append(joey_val)
    val_data.append(monica_val)
    val_data.append(phoebe_val)
    val_data.append(rachel_val)
    val_data.append(ross_val)
    chandler_loss_val = []
    joey_loss_val = []
    monica_loss_val = []
    phoebe_loss_val = []
    rachel_loss_val = []
    ross_loss_val = []
    minimum_samples = min_samples(val_data)
    max_range = np.multiply(int(minimum_samples/batch_size),batch_size)
    for i in range(0,max_range,batch_size):
        for j in range(len(val_data)):
            X_val = val_data[j]
            X_utter1 = X_val[:,0]
            X_utter2 = X_val[:,1]
            X_p_vad = X_val[:,3:6]
            X_emo_vad = X_train[:,6:9]
            X_emo_labels = X_val[:,-7:]
            # select batch
            utter1 = X_utter1[i:i+batch_size]
            utter2 = X_utter2[i:i+batch_size]
            utter1 = normalize_text(utter1)
            utter2 = normalize_text(utter2)
            p_vad = X_p_vad[i:i+batch_size].astype('float32')
            cat_labels = X_emo_labels[i:i+batch_size].astype('float32')
            vad_labels = X_emo_vad[i:i+batch_size].astype('float32')
            # convert into tensors
            utter1 = tf.convert_to_tensor(utter1)
            utter2 = tf.convert_to_tensor(utter2)
            p_vad = tf.convert_to_tensor(p_vad)
            cat_labels = tf.convert_to_tensor(cat_labels)
            vad_labels = tf.convert_to_tensor(vad_labels)
            # bert encoder
            utter1_sementics = bert_encoder(utter1)
            utter2_sementics = bert_encoder(utter2)
            dialogue_sementics = tf.concat([utter1_sementics, utter2_sementics],1)
            if j == 0:
                # personality vad
                per_vad = personality_vad_1(vad_labels,p_vad,dialogue_sementics,training=False)
                # logits
                logits = model_1(per_vad,training=False)
                # loss function  
                loss_value = loss_fn(cat_labels, logits)
                chandler_loss_val.append(loss_value)
            elif j == 1:
                # subjective output
                per_vad = personality_vad_2(vad_labels,p_vad,dialogue_sementics,training=False)
                # logits
                logits = model_2(per_vad,training=False)
                # loss function  
                loss_value = loss_fn(cat_labels, logits)
                joey_loss_val.append(loss_value)
            elif j == 2:
                # subjective output
                per_vad = personality_vad_3(vad_labels,p_vad,dialogue_sementics,training=False)
                # logits
                logits = model_3(per_vad,training=False)
                # loss function  
                loss_value = loss_fn(cat_labels, logits)
                monica_loss_val.append(loss_value)
            elif j == 3:
                # subjective output
                per_vad = personality_vad_4(vad_labels,p_vad,dialogue_sementics,training=False)
                # logits
                logits = model_4(per_vad,training=False)
                # loss function  
                loss_value = loss_fn(cat_labels, logits)
                phoebe_loss_val.append(loss_value)
            elif j == 4:
                # subjective output
                per_vad = personality_vad_5(vad_labels,p_vad,dialogue_sementics,training=False)
                # logits
                logits = model_5(per_vad,training=False)
                # loss function  
                loss_value = loss_fn(cat_labels, logits)
                rachel_loss_val.append(loss_value)
            elif j == 5:
                # subjective output
                per_vad = personality_vad_6(vad_labels,p_vad,dialogue_sementics,training=False)
                # logits
                logits = model_6(per_vad,training=False)
                # loss function  
                loss_value = loss_fn(cat_labels, logits)
                ross_loss_val.append(loss_value)
    print('-' * 20)
    print('Validation Loss:')
    print('-' * 20)
    print('Chandler Loss:', np.mean(chandler_loss_val))
    print('Joey Loss:', np.mean(joey_loss_val))
    print('Monica Loss:', np.mean(monica_loss_val))
    print('Phoebe Loss:', np.mean(phoebe_loss_val))
    print('Rachel Loss:', np.mean(rachel_loss_val))
    print('Ross Loss:', np.mean(ross_loss_val))
    #if epoch == 0:
    #if (epoch+1) % 5 == 0:      
    # test the model
    # concatenate all test samples
    chandler_logits = []
    chandler_labels = []
    joey_logits = []
    joey_labels = []
    monica_logits = []
    monica_labels = []
    phoebe_logits = []
    phoebe_labels = []
    rachel_logits = []
    rachel_labels = []
    ross_logits = []
    ross_labels = []
    test_data = []
    test_data.append(chandler_test)
    test_data.append(joey_test)
    test_data.append(monica_test)
    test_data.append(phoebe_test)
    test_data.append(rachel_test)
    test_data.append(ross_test)
    minimum_samples = min_samples(test_data)
    max_range = np.multiply(int(minimum_samples/batch_size),batch_size)
    for i in range(0,max_range,batch_size):
      for j in range(len(test_data)):
        X_test = test_data[j]
        X_utter1 = X_test[:,0]
        X_utter2 = X_test[:,1]
        X_p_vad = X_test[:,3:6]
        X_emo_vad = X_train[:,6:9]
        X_emo_labels = X_test[:,-7:]
        # select batch
        utter1 = X_utter1[i:i+batch_size]
        utter2 = X_utter2[i:i+batch_size]
        p_vad = X_p_vad[i:i+batch_size].astype('float32')
        cat_labels = X_emo_labels[i:i+batch_size].astype('float32')
        vad_labels = X_emo_vad[i:i+batch_size].astype('float32')
        # convert into tensors
        utter1 = tf.convert_to_tensor(utter1)
        utter2 = tf.convert_to_tensor(utter2)
        p_vad = tf.convert_to_tensor(p_vad)
        cat_labels = tf.convert_to_tensor(cat_labels)
        vad_labels = tf.convert_to_tensor(vad_labels)
        # bert encoder
        utter1_sementics = bert_encoder(utter1)
        utter2_sementics = bert_encoder(utter2)
        dialogue_sementics = tf.concat([utter1_sementics, utter2_sementics],1)
        if j == 0:
            # subjective output
            per_vad = personality_vad_1(vad_labels,p_vad,dialogue_sementics,training=False)
            # logits
            logits = model_1(per_vad,training=False)
            chandler_logits.append(logits)
            chandler_labels.append(cat_labels)
        elif j == 1:
            # subjective output
            per_vad = personality_vad_2(vad_labels,p_vad,dialogue_sementics,training=False)
            # logits
            logits = model_2(per_vad,training=False)
            joey_logits.append(logits)
            joey_labels.append(cat_labels)
        elif j == 2:
            # subjective output
            per_vad = personality_vad_3(vad_labels,p_vad,dialogue_sementics,training=False)
            # logits
            logits = model_3(per_vad,training=False)
            monica_logits.append(logits)
            monica_labels.append(cat_labels)
        elif j == 3:
            # subjective output
            per_vad = personality_vad_4(vad_labels,p_vad,dialogue_sementics,training=False)
            # logits
            logits = model_4(per_vad,training=False)
            phoebe_logits.append(logits)
            phoebe_labels.append(cat_labels)
        elif j == 4:
            # subjective output
            per_vad = personality_vad_5(vad_labels,p_vad,dialogue_sementics,training=False)
            # logits
            logits = model_5(per_vad,training=False)
            rachel_logits.append(logits)
            rachel_labels.append(cat_labels)
        elif j == 5:
            # subjective output
            per_vad = personality_vad_6(vad_labels,p_vad,dialogue_sementics,training=False)
            # logits
            logits = model_6(per_vad,training=False)
            ross_logits.append(logits)
            ross_labels.append(cat_labels)

    target_names = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']

    chandler_true,chandler_pred = reduce_dimension(chandler_labels,chandler_logits)
    chandler_result = classification_report(chandler_true,chandler_pred,  digits=4, output_dict=True,zero_division=1)

    joey_true,joey_pred = reduce_dimension(joey_labels,joey_logits)
    joey_result = classification_report(joey_true,joey_pred,  digits=4, output_dict=True,zero_division=1)

    monica_true,monica_pred = reduce_dimension(monica_labels,monica_logits)
    monica_result = classification_report(monica_true,monica_pred,  digits=4, output_dict=True,zero_division=1)

    phoebe_true,phoebe_pred = reduce_dimension(phoebe_labels,phoebe_logits)
    phoebe_result = classification_report(phoebe_true,phoebe_pred,  digits=4, output_dict=True,zero_division=1)

    rachel_true,rachel_pred = reduce_dimension(rachel_labels,rachel_logits)
    rachel_result = classification_report(rachel_true,rachel_pred,  digits=4, output_dict=True,zero_division=1)

    ross_true,ross_pred = reduce_dimension(ross_labels,ross_logits)
    ross_result = classification_report(ross_true,ross_pred,  digits=4, output_dict=True,zero_division=1)
    # chandler
    chandler_results_dict = {}
    for key in chandler_result.keys():
      if key != 'accuracy':
        f1_score = chandler_result.get(key, {}).get('f1-score')
        chandler_results_dict[key] = f1_score
    print('Chandler:')
    print(chandler_results_dict)
    # joey
    joey_results_dict = {}
    for key in joey_result.keys():
      if key != 'accuracy':
        f1_score = joey_result.get(key, {}).get('f1-score')
        joey_results_dict[key] = f1_score
    print('Joey:')
    print(joey_results_dict)
    # monica
    monica_results_dict = {}
    for key in monica_result.keys():
      if key != 'accuracy':
        f1_score = monica_result.get(key, {}).get('f1-score')
        monica_results_dict[key] = f1_score
    print('Monica:')
    print(monica_results_dict)
    # phoebe
    phoebe_results_dict = {}
    for key in phoebe_result.keys():
      if key != 'accuracy':
        f1_score = phoebe_result.get(key, {}).get('f1-score')
        phoebe_results_dict[key] = f1_score
    print('Phoebe:')
    print(phoebe_results_dict)
    # rachel
    rachel_results_dict = {}
    for key in rachel_result.keys():
      if key != 'accuracy':
        f1_score = rachel_result.get(key, {}).get('f1-score')
        rachel_results_dict[key] = f1_score
    print('Rachel:')
    print(rachel_results_dict)
    # ross
    ross_results_dict = {}
    for key in ross_result.keys():
      if key != 'accuracy':
        f1_score = ross_result.get(key, {}).get('f1-score')
        ross_results_dict[key] = f1_score
    print('Ross:')
    print(ross_results_dict)
    average_dict = {}
    for Key1,values1 in chandler_results_dict.items():
      for Key2,values2 in joey_results_dict.items():
        for Key3,values3 in monica_results_dict.items():
          for Key4,values4 in phoebe_results_dict.items():
              for Key5,values5 in rachel_results_dict.items():
                  for Key6,values6 in ross_results_dict.items():
                      if Key1 == Key2 == Key3 == Key4 == Key5 == Key6:
                          add_val = values1+values2+values3+values4+values5+values6
                          average_dict[Key1] = (add_val/6)

    print('-' * 20)
    print('Average F1-score of all the emotion categories w.r.t each character:')
    print('-' * 20)
    print(average_dict)

